In [ ]:
!pip install ludwig
!pip install --upgrade albumentations
!pip install --upgrade pandas
!pip install --upgrade numpy
!pip install --upgrade scikit-image
!pip install --upgrade https://github.com/uber/ludwig/archive/master.zip --no-dependencies

In [ ]:
import pandas as pd
df = pd.read_csv("../input/train/train.csv")
df['AdoptionSpeed2'] = df.AdoptionSpeed
df.to_csv("train_copy.csv")
df = pd.read_csv("../input/test/test.csv")
df['AdoptionSpeed2'] = df.Age
df.to_csv("test_copy.csv")

In [ ]:
with open("./model_def_multi_text.yml", "w+") as f:
    model = """input_features:
    -
        name: Description
        type: text
        encoder: parallel_cnn
        level: char
        dropout: true
    -
        name: Type
        type: binary
    -
        name: Age
        type: numerical
    -
        name: Breed1
        type: category
    -
        name: Breed2
        type: category
    -
        name: Gender
        type: category
    -
        name: Color1
        type: category
    -
        name: Color2
        type: category
    -
        name: Color3
        type: category
    -
        name: MaturitySize
        type: category
    -
        name: FurLength
        type: category
    -
        name: Vaccinated
        type: category
    -
        name: Dewormed
        type: category
    -
        name: Sterilized
        type: category
    -
        name: Health
        type: category
    -
        name: Quantity
        type: numerical
    -
        name: Fee
        type: numerical
    -
        name: State
        type: category
    -
        name: VideoAmt
        type: numerical
    -
        name: PhotoAmt
        type: numerical
        
combiner:
    type: concat
    num_fc_layers: 2
    fc_size: 256
    dropout: true
        
output_features:
    -
        name: AdoptionSpeed
        type: numerical
    -
        name: AdoptionSpeed2
        type: category
        
preprocessing:
    stratify: AdoptionSpeed2
        
training:
    dropout_rate: 0.5
        """
    f.write(model)

In [ ]:
!ludwig experiment \
  --data_csv train_copy.csv \
  --model_definition_file model_def_multi_text.yml

In [ ]:
!ludwig train \
  --data_csv train_copy.csv \
  --model_definition_file model_def_multi_text.yml

See the error below:

In [ ]:
!ludwig predict --only_predictions\
  --data_csv ../input/test/test.csv \
  --model_path results/experiment_run_1/model/

Here I continue with the workaround:

In [ ]:
!ludwig predict --only_predictions\
  --data_csv ./test_copy.csv \
  --model_path results/experiment_run_1/model/

In [ ]:
!head results_0/AdoptionSpeed2_predictions.csv

In [ ]:
!head ../input/test/sample_submission.csv

In [ ]:
preds = pd.read_csv("results_0/AdoptionSpeed2_predictions.csv", header=None)
df = pd.read_csv("../input/test/test.csv")
df['AdoptionSpeed'] = preds

In [ ]:
submission = pd.DataFrame({'PetID': df.PetID, 'AdoptionSpeed':df.AdoptionSpeed},
                         columns = ['PetID', 'AdoptionSpeed'])
submission.to_csv("submission.csv", index=False)

In [ ]:
!head submission.csv